In [1]:
import sys
sys.path.insert(0, '/home/jovyan/Proj/ml_combiner/TSEBRA/bin')
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
import matplotlib.pyplot as plt
from genome_anno import Anno
from overlap_graph import Graph
from evidence import Evidence
from gnn import GNN, get_inputs
import pickle
import tensorflow as tf
from copy import deepcopy
import time
#physical_devices = tf.config.list_physical_devices('GPU') 
#for device in physical_devices:
    #tf.config.experimental.set_memory_growth(device, True)

In [2]:
"""input_train[0][0]['input_nodes'][0][2].shape
a=input_train[0]
#a[0]['input_nodes'][0] = np.append(a[0]['input_nodes'], input_train[1][0]['input_nodes'])

np.concatenate((a[0]['input_nodes'][0], input_train[1][0]['input_nodes'][0])).shape
print(a[0]['incidence_matrix_sender'][0])
print(input_train[31][0]['incidence_matrix_sender'][0])
shape1=a[0]['incidence_matrix_sender'][0].shape
shape2=input_train[31][0]['incidence_matrix_sender'][0].shape
np.append(a[0]['incidence_matrix_sender'][0], np.zeros((shape2[0],shape1[1]), bool), axis=0)
np.append(np.zeros((shape1[0],shape2[1]), bool), input_train[31][0]['incidence_matrix_sender'][0], axis=0)"""

"input_train[0][0]['input_nodes'][0][2].shape\na=input_train[0]\n#a[0]['input_nodes'][0] = np.append(a[0]['input_nodes'], input_train[1][0]['input_nodes'])\n\nnp.concatenate((a[0]['input_nodes'][0], input_train[1][0]['input_nodes'][0])).shape\nprint(a[0]['incidence_matrix_sender'][0])\nprint(input_train[31][0]['incidence_matrix_sender'][0])\nshape1=a[0]['incidence_matrix_sender'][0].shape\nshape2=input_train[31][0]['incidence_matrix_sender'][0].shape\nnp.append(a[0]['incidence_matrix_sender'][0], np.zeros((shape2[0],shape1[1]), bool), axis=0)\nnp.append(np.zeros((shape1[0],shape2[1]), bool), input_train[31][0]['incidence_matrix_sender'][0], axis=0)"

In [3]:
def set_weight():
    global weight_class_one
    a=0
    b=0
    for i in input_train:    
        a+=1
        for k in list(i[1].values())[0]:        
            a += k.shape[0]
            b += np.sum(k[:,0])
    weight_class_one = (a-b)/b
    print(a, b, weight_class_one)

In [4]:
"""
def prep_inputs():
    global input_train
    i_components = deepcopy(input_components)
    np.random.shuffle(i_components)
    input_train = [i_components[0]]    
    for comp in i_components[1:]:
        #print(comp[0]['input_edges'][0].shape, comp[0]['input_edges'][0].size)
        #print(len(input_train), len(input_train[-1][0]['input_nodes'][0]), len(comp[0]['input_nodes'][0]))
        if len(input_train[-1][0]['input_nodes'][0]) < batch_size:            
            for type in ['input_nodes', 'input_edges']:
                if comp[0][type][0].size > 0:
                    #print(input_train[-1][0][type][0].shape, comp[0][type][0].shape)
                    #print(comp[0][type][0].shape)
                    input_train[-1][0][type] = np.expand_dims(np.concatenate((input_train[-1][0][type][0], comp[0][type][0])),0)
                    #print(comp[0][type][0].shape)
            
            
            for type in ['incidence_matrix_sender', 'incidence_matrix_receiver']:
                shape1 = input_train[-1][0][type][0].shape
                if comp[0]['input_edges'][0].size>0:                    
                    shape2 = comp[0][type][0].shape
                    input_train[-1][0][type] = tf.expand_dims(                    
                        np.append( np.append(input_train[-1][0][type][0], np.zeros((shape2[0], shape1[1])), axis=0),
                              np.append(comp[0][type][0], np.zeros((shape1[0], shape2[1])), axis=0),
                            axis=1),0)
                else:
                    input_train[-1][0][type] = tf.expand_dims(                    
                        np.append(input_train[-1][0][type][0], 
                                  np.zeros((comp[0]['input_nodes'][0].shape[0], shape1[1])), axis=0),0)
            type = 'target_label'
            shape1 = input_train[-1][1][type][0].shape
            shape2 = comp[1][type][0].shape
                                
            t = np.append( np.append(input_train[-1][1][type][0], np.zeros((shape2[0], shape1[1])), axis=0),
                      np.append(comp[1][type][0][:,1:], np.zeros((shape1[0], shape2[1]-1)), axis=0),
                    axis=1)
            t[shape1[0]:,0] = comp[1][type][0][:,0]
            input_train[-1][1][type] = tf.expand_dims(t,0)
        else:
            input_train.append(comp)
def prep_inputs3():
    global input_train
    
    np.random.shuffle(input_components)    
    input_train = [[{'input_nodes' : [], 'input_edges' : [], 
                     'incidence_matrix_sender' : [], 'incidence_matrix_receiver': []}, {'target_label' : []}]]
    for k, comp in enumerate(input_components):
        for t in input_train[-1][0].keys():
            input_train[-1][0][t].append(np.array(comp[0][t]))
        input_train[-1][1]['target_label'].append(comp[1]['target_label'])
        if (k+1)%batch_size == 0:
            for t in input_train[-1][0].keys():
                #print(input_train[-1][0][t])
                input_train[-1][0][t] = tf.ragged.constant(input_train[-1][0][t])
            input_train[-1][1]['target_label'] = tf.ragged.constant(input_train[-1][1]['target_label'])
            input_train.append([{'input_nodes' : [], 'input_edges' : [], 
                     'incidence_matrix_sender' : [], 'incidence_matrix_receiver': []}, {'target_label' : []}])
        """
        

"\ndef prep_inputs():\n    global input_train\n    i_components = deepcopy(input_components)\n    np.random.shuffle(i_components)\n    input_train = [i_components[0]]    \n    for comp in i_components[1:]:\n        #print(comp[0]['input_edges'][0].shape, comp[0]['input_edges'][0].size)\n        #print(len(input_train), len(input_train[-1][0]['input_nodes'][0]), len(comp[0]['input_nodes'][0]))\n        if len(input_train[-1][0]['input_nodes'][0]) < batch_size:            \n            for type in ['input_nodes', 'input_edges']:\n                if comp[0][type][0].size > 0:\n                    #print(input_train[-1][0][type][0].shape, comp[0][type][0].shape)\n                    #print(comp[0][type][0].shape)\n                    input_train[-1][0][type] = np.expand_dims(np.concatenate((input_train[-1][0][type][0], comp[0][type][0])),0)\n                    #print(comp[0][type][0].shape)\n            \n            \n            for type in ['incidence_matrix_sender', 'incidence_matrix_

In [5]:
"""input_components = np.array(input_components)
print(len(input_components[1][0]['input_nodes']),
     len(input_train[0][0]['input_edges']))"""

"input_components = np.array(input_components)\nprint(len(input_components[1][0]['input_nodes']),\n     len(input_train[0][0]['input_edges']))"

In [6]:
"""print(input_components[1][1]['target_label'])
batch_size=3
print(batch_size)
input_components = np.array(input_components)
start_time = time.time()
#prep_inputs()
print("--- %s seconds ---" % (time.time() - start_time))
start_time = time.time()
prep_inputs2()
print("--- %s seconds ---" % (time.time() - start_time))
print(max([len(n.nodes) for n in g.batches]))
print(input_train[0][0]['input_nodes'][0].shape)
print(input_components[1][0]['input_nodes'][0].shape)
print(max([len(i[0]['input_nodes'][0]) for i in input_components]))"""

'print(input_components[1][1][\'target_label\'])\nbatch_size=3\nprint(batch_size)\ninput_components = np.array(input_components)\nstart_time = time.time()\n#prep_inputs()\nprint("--- %s seconds ---" % (time.time() - start_time))\nstart_time = time.time()\nprep_inputs2()\nprint("--- %s seconds ---" % (time.time() - start_time))\nprint(max([len(n.nodes) for n in g.batches]))\nprint(input_train[0][0][\'input_nodes\'][0].shape)\nprint(input_components[1][0][\'input_nodes\'][0].shape)\nprint(max([len(i[0][\'input_nodes\'][0]) for i in input_components]))'

In [7]:
class SampleGenerator(keras.utils.Sequence):
    def __init__(self, val, epoch_len):
        self.val = val
        self.epoch_len = epoch_len
        self.shuffle=True
        self.on_epoch_end()

    def __len__(self):
        return self.epoch_len #number of gradient descent steps per epoch

    def on_epoch_end(self):
        global input_train
        'Updates indexes after each epoch'
        #prep_inputs3()
        #self.epoch_len = len(input_train)-1
        #self.indexes = np.arange(self.epoch_len)
        #input_train = []
        #for g in graph_list:            
            #i, _, _, _, _ = get_batches(g, int(numb_batches_train / len(train_species)), 
                                        #batch_size, int(numb_batches_train_no_edge / len(train_species)),0)
            #input_train += i
        #if self.shuffle == True:
            #np.random.shuffle(self.indexes)
            
    def __getitem__(self, _index):
        #i = self.indexes[_index]
        if self.val:
            return input_val[_index][0], input_val[_index][1]
        else:
            return input_train[_index][0], input_train[_index][1]

In [8]:
train_species = ['Populus_trichocarpa','Drosophila_melanogaster', 
                    'Medicago_truncatula', 'Caenorhabditis_elegans']
#train_species = ['Caenorhabditis_elegans', 'Drosophila_melanogaster', 
                 #'Medicago_truncatula']
#train_species= ['Drosophila_melanogaster']
val_species = ['Arabidopsis_thaliana', 'Danio_rerio'
               ]
#val_species= ['Drosophila_melanogaster']

quiet = True
numb_batches_train = 150 * len(train_species)
numb_batches_val = 120 *len(val_species)
numb_batches_train_no_edge = 75 * len(train_species)
numb_batches_val_no_edge = 75 *len(val_species)
numb_nodes_per_species = 17000
batch_size = 256#128
v=0
brain_dir = '/home/jovyan/brain/'
parent_dir = '/home/jovyan/Proj/ml_combiner/data/train_data'
out = f'{parent_dir}/train2/train4'
weight_class_one = 1.

In [9]:
def get_edge_max():
    e_f1 = []
    e_f2 = []
    for g in graph_list:        
        for edge in g.edges.values():            
            e_f1.append(edge.feature_vector_n1_to_n2)
            e_f2.append(edge.feature_vector_n2_to_n1)           
            
    e_f1 = np.array(e_f1)[:,:120]
    e_f2 = np.array(e_f2)[:,:120]
    print(e_f1.shape)
    print(e_f2.shape)
    print('EDGES ni to nj')
    print('MEAN\n', [i for i in np.mean(e_f1, axis=0)])
    print('STD\n',[i for i in np.std(e_f1, axis=0)])
    print('MAX\n',[i for i in np.max(e_f1, axis=0)])
    print('MIN\n',[i for i in np.min(e_f1, axis=0)])
    print('\nEDGES nj to ni')
    print('MEAN\n', [i for i in np.mean(e_f2, axis=0)])
    print('STD\n',[i for i in np.std(e_f2, axis=0)])
    print('MAX\n',[i for i in np.max(e_f2, axis=0)])
    print('MIN\n',[i for i in np.min(e_f2, axis=0)])
    
def get_node_max():
    n_f = []
    for g in graph_list:        
        for node in g.nodes.values():            
            n_f.append(node.feature_vector)     
            
    n_f = np.array(n_f)[:,:80]
    print(n_f.shape)
    print('NODES')
    print('MEAN\n', [i for i in np.mean(n_f, axis=0)])
    print('STD\n',[i for i in np.std(n_f, axis=0)])
    print('MAX\n',[i for i in np.max(n_f, axis=0)])
    print('MIN\n',[i for i in np.min(n_f, axis=0)])
    print('MIN\n',[i for i in np.min(n_f, axis=0)])

In [10]:
def describe_features(graph):
    n_f = []    
    n_f_true = []
    e_f_tt = []
    e_f_tf = []
    e_f_ft = []
    e_f_ff = []
    
#     for node in graph.nodes.values():
#         n_f.append(node.feature_vector)
#     for edge in graph.edges.values():
#         e_f.append(edge.feature_vector_n1_to_n2)
#         e_f.append(edge.feature_vector_n2_to_n1)
    for g in graph_list:
        for node in g.nodes.values():
            if node.is_in_ref_anno:
                n_f_true.append(node.feature_vector)
            else:
                n_f.append(node.feature_vector)
        for edge in g.edges.values():
            n1 = g.nodes[edge.node1]
            n2 = g.nodes[edge.node2]
            if n1.is_in_ref_anno and n2.is_in_ref_anno:                
                e_f_tt.append(edge.feature_vector_n1_to_n2)
                e_f_tt.append(edge.feature_vector_n2_to_n1)
            elif n1.is_in_ref_anno and not n2.is_in_ref_anno:
                e_f_tf.append(edge.feature_vector_n1_to_n2)
                e_f_ft.append(edge.feature_vector_n2_to_n1)
            elif not n1.is_in_ref_anno and n2.is_in_ref_anno:
                e_f_ft.append(edge.feature_vector_n1_to_n2)
                e_f_tf.append(edge.feature_vector_n2_to_n1)
            elif not n1.is_in_ref_anno and not n2.is_in_ref_anno:
                e_f_ff.append(edge.feature_vector_n1_to_n2)
                e_f_ff.append(edge.feature_vector_n2_to_n1)
            
    n_f = np.array(n_f)
    n_f_true = np.array(n_f_true)
    print(n_f.shape)
    print(n_f_true.shape)
    e_f_ff = np.array(e_f_ff)
    e_f_tt = np.array(e_f_tt)
    e_f_tf = np.array(e_f_tf)
    e_f_ft = np.array(e_f_ft)
    print(e_f_tt.shape)
    print(e_f_ft.shape)
    print(e_f_tf.shape)
    print(e_f_ff.shape)
    
    n_f_all = np.concatenate((n_f,n_f_true))
    print(n_f_all.shape)
    e_f_all = np.concatenate((e_f_ff,e_f_tt,e_f_tf,e_f_ft))
    print('NODES')
    print('MEAN\n', [i for i in np.mean(n_f_all, axis=0)])
    print('STD\n', [i for i in np.std(n_f_all, axis=0)])
    print('MAX\n', [i for i in np.max(n_f_all, axis=0)])
    print('Min\n', [i for i in np.min(n_f_all, axis=0)])
    print('EDGES')
    print('MEAN\n', [i for i in np.mean(e_f_all, axis=0)])
    print('STD\n',[i for i in np.std(e_f_all, axis=0)])
    print('MAX\n',[i for i in np.max(e_f_all, axis=0)])
    print('MIN\n',[i for i in np.min(e_f_all, axis=0)])
    
    fig, ax = plt.subplots(ncols = 1, nrows=4, figsize=(15,9))
    i = int(n_f.shape[1] / 4 + 0.5)
    for j in range(4):
        data = [n_f[:,k] for k in range(i*j, min(i*(j+1), n_f.shape[1]))]
        data2 = [n_f_true[:,k] for k in range(i*j, min(i*(j+1), n_f_true.shape[1]))]
        bp = ax[j].boxplot(data, positions=np.array(range(len(data)))*2.0-0.4, sym='', widths=0.6)
        for item in ['boxes', 'medians']:            
            plt.setp(bp[item], color='red')
        bp = ax[j].boxplot(data2, positions=np.array(range(len(data2)))*2.0+0.4, sym='', widths=0.6)
        for item in ['boxes', 'medians']:            
            plt.setp(bp[item], color='green')
        ax[j].set_ylim([-3,3])
        ax[j].set_xticks([])
        
    plt.savefig(brain_dir + '/test_data/ml_training/plot/node_features.png', dpi=200)
    fig.suptitle(f'node features')
    fig, ax = plt.subplots(ncols = 1, nrows=4, figsize=(15,9))
    i = int(e_f_tt.shape[1] / 4 + 0.5)
    for j in range(4):
        for c, e_f, v in zip(['red', 'orange', 'violet', 'green'], [e_f_ff, e_f_ft, e_f_tf, e_f_tt], 
                            [1.1, 0.4, -0.4, -1.1]):
            data = [e_f[:,k] for k in range(i*j, min(i*(j+1), e_f.shape[1]))]
            bp = ax[j].boxplot(data, positions=np.array(range(len(data)))*4.0-v, sym='', widths=0.6)
            for item in ['boxes', 'medians']:            
                plt.setp(bp[item], color=c)
        ax[j].set_ylim([-3,3])
    fig.suptitle(f'edge features')
    plt.savefig(brain_dir + '/test_data/ml_training/plot/edge_features.png', dpi=200)
    plt.show()

In [11]:
"""def get_batches(tx_sets, evi, anno, numb_b, b_size, v_size, des=False):
    graph = Graph(tx_sets, verbose=v)
    graph.build()
    graph.add_node_features(evi)
    graph.add_edge_features(evi)
    graph.add_reference_anno_label(anno)
    graph.create_batch(numb_b, b_size, repl=False)    
    train, val = graph.get_batches_as_input_target(v_size)
    if des:
        describe_features(graph)
    return train, val, graph"""

def get_batches(graph, numb_b, b_size, numb_b_n_e, v_size, des=False):    
    graph.create_batch(numb_b, b_size, repl=False)  
    #graph.create_batch_no_edges(numb_b_n_e, b_size, repl=False)
    train, val, train_no_edge, val_no_edge = graph.get_batches_as_input_target(v_size)
    if des:
        describe_features(graph)
    return train, val, train_no_edge, val_no_edge, graph

In [12]:
"""c = 1
gene_sets = []
val_sets = []
for b in ['braker1', 'braker2']:        
    gene_sets.append(Anno(f'{parent_dir}/{b}_train.gtf', f'anno{c}'))
    gene_sets[-1].addGtf()
    gene_sets[-1].norm_tx_format()
    val_sets.append(Anno(f'{parent_dir}/{b}_val.gtf', f'anno{c}'))
    val_sets[-1].addGtf()
    val_sets[-1].norm_tx_format()

    c += 1

ref_anno = Anno(f'{parent_dir}/annot_train.gtf', 'reference')
ref_anno.addGtf()
ref_anno.norm_tx_format()
ref_anno_val = Anno(f'{parent_dir}/annot_val.gtf', 'reference')
ref_anno_val.addGtf()
ref_anno_val.norm_tx_format()

# read hintfiles
evi_train = Evidence()
for h in ['hints1', 'hints2']:    
    evi_train.add_hintfile(f'{parent_dir}/{h}_train.gff')
    
evi_val = Evidence()
for h in ['hints1', 'hints2']:        
    evi_val.add_hintfile(f'{parent_dir}/{h}_val.gff')"""


"c = 1\ngene_sets = []\nval_sets = []\nfor b in ['braker1', 'braker2']:        \n    gene_sets.append(Anno(f'{parent_dir}/{b}_train.gtf', f'anno{c}'))\n    gene_sets[-1].addGtf()\n    gene_sets[-1].norm_tx_format()\n    val_sets.append(Anno(f'{parent_dir}/{b}_val.gtf', f'anno{c}'))\n    val_sets[-1].addGtf()\n    val_sets[-1].norm_tx_format()\n\n    c += 1\n\nref_anno = Anno(f'{parent_dir}/annot_train.gtf', 'reference')\nref_anno.addGtf()\nref_anno.norm_tx_format()\nref_anno_val = Anno(f'{parent_dir}/annot_val.gtf', 'reference')\nref_anno_val.addGtf()\nref_anno_val.norm_tx_format()\n\n# read hintfiles\nevi_train = Evidence()\nfor h in ['hints1', 'hints2']:    \n    evi_train.add_hintfile(f'{parent_dir}/{h}_train.gff')\n    \nevi_val = Evidence()\nfor h in ['hints1', 'hints2']:        \n    evi_val.add_hintfile(f'{parent_dir}/{h}_val.gff')"

In [13]:
#input_train, _, g1 = get_batches(gene_sets, evi_train, ref_anno, numb_batches_train, batch_size, 0)
#input_val, _, g2 = get_batches(val_sets, evi_val, ref_anno_val, numb_batches_val, batch_size, 0) 
input_train = []
input_components = []
input_train_no_edge = []
input_val = []
input_val_no_edge = []
graph_list = []
train_components=[]
val_components=[]
for s in train_species:
    #with open(f'{brain_dir}/test_data/ml_training/train_data/{s}_graph_object.pkl', 'rb') as inp:
    with open(f'{brain_dir}/test_data/ml_training/train_data/{s}_altseqs_graph_object.pkl', 'rb') as inp:
        g = pickle.load(inp)
        graph_list.append(g)
    #i, _, j, _, _ = get_batches(g, int(numb_batches_train / len(train_species)), 
                                #batch_size, int(numb_batches_train_no_edge / len(train_species)),0)
    i, _ = g.get_components_as_input_target(numb_nodes_per_species, 0, True)    
    train_components += i
    #input_train_no_edge += j

for s in val_species:
    #with open(f'{brain_dir}/test_data/ml_training/train_data/{s}_graph_object.pkl', 'rb') as inp:
    with open(f'{brain_dir}/test_data/ml_training/train_data/{s}_altseqs_graph_object.pkl', 'rb') as inp:
        g = pickle.load(inp)
    #i, _, j, _, _ = get_batches(g, int(numb_batches_val / len(val_species)), 
                                #batch_size, numb_batches_val_no_edge, 0)
    
    i, _ = g.get_components_as_input_target(int(numb_nodes_per_species/2), 1, True)
    val_components += i
    #input_val_no_edge += j


17000
17002
17001
17000
8500
8503


In [14]:
#describe_features(graph_list[0])
#get_node_max()
#get_edge_max()

In [15]:
input_train = get_inputs(train_components, batch_size, False)
input_val = get_inputs(val_components, batch_size, False)
train_gen = SampleGenerator(False, len(input_train))#int(numb_batches * (1-val_size)))
val_gen = SampleGenerator(True, len(input_val))

In [16]:
set_weight()
gnn = GNN(weight_class_one=weight_class_one)
gnn.compile(out)

68266 17883.0 2.8173684504836998


In [ ]:
NUM_EPOCHS = 5000
#history = gnn.train(train_gen, val_gen, NUM_EPOCHS, out) #!
history = gnn.train(train_gen, val_gen, NUM_EPOCHS, out)

history.history["last_iteration_binary_accuracy"][-1]
_, ax = plt.subplots(ncols = 2, figsize=(15,9))

ax[0].plot(np.arange(NUM_EPOCHS), history.history["loss"], 'b', label = 'Training loss')
ax[0].plot(np.arange(NUM_EPOCHS), history.history["val_loss"], 'g', label = 'Validation loss')
ax[0].set_title('Training and validation loss')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].legend()

ax[1].plot(np.arange(NUM_EPOCHS), history.history["last_iteration_binary_accuracy"], 'b', label = 'Training accuracy')
ax[1].plot(np.arange(NUM_EPOCHS), history.history["val_last_iteration_binary_accuracy"], 'g', label = 'Validation accuracy')
ax[1].set_title('Training and validation accuracy of the last message passing iteration')
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('loss')
ax[1].legend()

plt.savefig(out + '.png', dpi=200)

Epoch 1/5000
263/263 [==============================] - 7s 18ms/step - loss: 0.2163 - last_iteration_binary_accuracy: 0.9798 - val_loss: 2.8727 - val_last_iteration_binary_accuracy: 0.9702
Epoch 2/5000
263/263 [==============================] - 4s 17ms/step - loss: 0.2158 - last_iteration_binary_accuracy: 0.9668 - val_loss: 2.9402 - val_last_iteration_binary_accuracy: 0.9663
Epoch 3/5000
263/263 [==============================] - 5s 18ms/step - loss: 0.2168 - last_iteration_binary_accuracy: 0.9651 - val_loss: 2.8826 - val_last_iteration_binary_accuracy: 0.9650
Epoch 4/5000
263/263 [==============================] - 5s 18ms/step - loss: 0.2179 - last_iteration_binary_accuracy: 0.9645 - val_loss: 2.8928 - val_last_iteration_binary_accuracy: 0.9644
Epoch 5/5000
263/263 [==============================] - 5s 18ms/step - loss: 0.2191 - last_iteration_binary_accuracy: 0.9639 - val_loss: 2.8834 - val_last_iteration_binary_accuracy: 0.9639
Epoch 6/5000
263/263 [==============================] -

In [ ]:
gnn.predict(input_train[100][0])

In [ ]:
"""input_train = input_train_no_edge
input_val = input_val_no_edge
train_gen = SampleGenerator(False, len(input_train))#int(numb_batches * (1-val_size)))
val_gen = SampleGenerator(True, len(input_val))#int(numb_batches * val_size))

NUM_EPOCHS = 500
history = gnn.train_nn(train_gen, val_gen, NUM_EPOCHS, out) #!
    
history.history["last_iteration_binary_accuracy"][-1]
_, ax = plt.subplots(ncols = 2, figsize=(15,9))

ax[0].plot(np.arange(NUM_EPOCHS), history.history["loss"], 'b', label = 'Training loss')
ax[0].plot(np.arange(NUM_EPOCHS), history.history["val_loss"], 'g', label = 'Validation loss')
ax[0].set_title('Training and validation loss')
ax[0].set_xlabel('epoch')
ax[0].set_ylabel('loss')
ax[0].legend()

ax[1].plot(np.arange(NUM_EPOCHS), history.history["last_iteration_binary_accuracy"], 'b', label = 'Training accuracy')
ax[1].plot(np.arange(NUM_EPOCHS), history.history["val_last_iteration_binary_accuracy"], 'g', label = 'Validation accuracy')
ax[1].set_title('Training and validation accuracy of the last message passing iteration')
ax[1].set_xlabel('epoch')
ax[1].set_ylabel('loss')
ax[1].legend()

plt.savefig(out + '.png', dpi=200)"""